In [1]:
%pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras 3.5.0 requires ml-dtypes, which is not installed.
tensorflow-intel 2.17.0 requires ml-dtypes<0.5.0,>=0.3.1, which is not installed.
tensorflow-intel 2.17.0 requires tensorboard<2.18,>=2.17, which is not installed.
crewai-tools 0.51.1 requires embedchain>=0.1.114, which is not installed.
streamlit 1.39.0 requires protobuf<6,>=3.20, but you have protobuf 6.31.1 which is incompatible.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.


In [ ]:
from dotenv import load_dotenv()
load_dotenv()

In [ ]:
%pip uninstall pinecone-client

In [ ]:
%pip install pinecone

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [ ]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"

api_key=os.getenv("PINECONE_API_KEY")

## Initialising the pinecone client
pc=Pinecone(api_key=api_key)

## Creating the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, ## Dimension of the dense vector
        metric='dotproduct',
        spec=ServerlessSpec(cloud='aws',region="us-east-1")
    )

Exception: The official Pinecone python package has been renamed from `pinecone-client` to `pinecone`. Please remove `pinecone-client` from your project dependencies and add `pinecone` instead. See the README at https://github.com/pinecone-io/pinecone-python-client for more information on using the python SDK.

In [ ]:
index=pc.Index(index_name)
index

In [ ]:
## Vector embedding
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

In [ ]:
## Sparse matrix
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

In [ ]:
sentences=[
    "In 2022, I visited New York",
    "In 2021,I visited New Orleans"
]

## tfidf va;ues of the sentences
bm25_encoder.fit(sentences)

## store the values to a jsoan fie
bm25_encoder.dump("bm25_values.json")

In [ ]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [ ]:
retriever.add_texts(
    [
    "In 2022, I visited New York",
    "In 2021,I visited New Orleans"
]
)

In [ ]:
retriever.invoke("what city did i visit last")